Basado en: 
https://medium.com/@gayani.parameswaran/q-a-chatbot-using-langchain-hugging-face-and-mistral-31cd576779ef

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import yaml

In [3]:
with open("keys.yaml") as keys:
    try:
        api_keys = yaml.safe_load(keys)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_keys["hf_model"]

In [5]:
## Read the pdfs from the folder
loader=PyPDFDirectoryLoader("./data/wikipedia")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
len(final_documents) #length --> 316

8424

In [6]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

c:\Users\feder\anaconda3\envs\nlp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\feder\anaconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\feder\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administr

In [ ]:
query="Which year was Airbnb founded?"
relevant_docments=vectorstore.similarity_search(query)
print(relevant_docments[0].page_content)

In [8]:
from langchain_community.utilities import SQLDatabase
import os
import pandas as pd
import yaml
import sqlite3
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

In [9]:
conn = sqlite3.connect("data/database.db")
tickers = pd.read_sql_query("SELECT distinct Security FROM master_ticker", conn)["Security"].tolist()
db = SQLDatabase.from_uri("sqlite:///data/database.db")
print(db.get_usable_table_names())

['income_statements', 'master_ticker', 'price_data']


In [12]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_keys["hf_model"]

llm = ChatHuggingFace(model = "HuggingFaceH4/zephyr-7b-beta")

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Which is the latest gross revenue of ACN?"})
response


ValidationError: 1 validation error for ChatHuggingFace
__root__
  Expected llm to be one of HuggingFaceTextGenInference, HuggingFaceEndpoint, HuggingFaceHub, HuggingFacePipeline received <class 'NoneType'> (type=type_error)

: 